### **TD 11 et 12 - Collecte, Traitement, et Analyse de données de réseaux sociaux**

*LEFEVRE Laura et LE CORRE Camille - LDD BI*

In [115]:
#Importation des modules

import json
import re
import pandas as pd
import os

#Module a installer
    #!pip install textblob
    #!pip install textblob-fr

On définit en premier temps, une fonction qui va nous permettre de récupérer dans une variable, la liste des dictionnaires représentant les différents tweets. 

In [114]:
def openFile(fileName):
    '''Fonction qui permet de lire et de recuperer un fichier json dans une variable data_dict'''
    
    with open(fileName, encoding='utf8') as json_data:
        data_dict = json.load(json_data)
        
        return data_dict

Par exemple, la commande suivante permet de récupérer dans la varible dic, la liste des tweets du fichier "versailles_tweets_100.json". On obtient une liste de dictionnaire et dic[0] nous permet de visualiser le premier tweet. On peut voir qu'il possède plusieurs clés qui ne vont pas nous servir. Nous allons donc pouvoir nettoyer chacun des tweets, en gardant les informations essentiels (id, text, hastags...). De plus, on va pouvoir nettoyer le texte du tweet en supprimant tous les caractères spéciaux.

In [116]:
dic = openFile("versailles_tweets_100.json")
dic[0]

{'_id': '1421616335700824064',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 1,
  'quote_count': 0},
 'id': '1421616335700824064',
 'conversation_id': '1421616335700824064',
 'author_id': '1339914264522461187',
 'text': 'Goumin des éléphants joueurs la même fatigue même 😫 #twitter225',
 'geo': {'place_id': '00b8943291443c8c'},
 'lang': 'fr',
 'created_at': '2021-07-31T23:38:41.000Z',
 'entities': {'hashtags': [{'start': 52, 'end': 63, 'tag': 'twitter225'}]}}

On code les fonctions permettant le nettoyage :

1. Supprimer les emojis

In [119]:
def remove_emojis(data):
    '''Fonction qui permet de supprimer les emojis du text d'un tweet 
    (source : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)'''
    
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

2. Supprimer tous les caractères spéciaux

In [120]:
def CleanText(text):
    '''Fonction qui permet de nettoyer le texte d'un tweet a l'aide de regex'''
    text = text.lower()
    text = remove_emojis(text)
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r'#([A-Za-z0-9_]+)', "", text)
    text = re.sub(r"[\,\!\?\%\(\)\/\"\&\+\#\$\£\%\:\.\@\-]", "", text)
    text = re.sub(r"[\é\è\ê]", "e", text)
    text = re.sub(r"[\ù]", "u", text)
    text = re.sub(r"[\à]", "a", text)
    text = re.sub(r"[\î\ï]", "i", text)
    text = re.sub(r"( )+", " ", text)
    text = re.sub(r"( )*$", "", text)
    text = re.sub(r"^( )", "", text)
    
    return text

Il faut maintenant créer la zone d'atterrissage des tweets. 
On crée donc une fonction qui va nous permettre d'ajouter les tweets dans un autre fichier json, semblable à l'initial, mais avec des textes de tweet nettoyés. 

In [88]:
def ZoneAtterrissage(tweet):
    '''Fonction qui permet de creer la zone d'atterrisage des tweets. Elle ajoute dans un fichier zone d'atterrissage, un tweet en le nettoyant.'''
    
    #Nettoie le texte du tweet et remplace le texte initiale par le texte nettoye
    tweet_clean = CleanText(tweet['text'])
    tweet['text'] = tweet_clean
    
    #Si le fichier de la zone d'atterrissage n'existe pas, on le creer et on y ajoute le tweet
    if os.path.exists("zone_atterrissage.json") == False:
        with open("zone_atterrissage.json", "w") as fil:
            json.dump([tweet], fil)
    
    #Sinon, on recupere les tweets deja presents, et on y ajoute celui en cours de traitement
    else :
        li_tweet = openFile("zone_atterrissage.json")
        with open("zone_atterrissage.json", 'w') as filout :
            li_tweet.append(tweet)
            json.dump(li_tweet, filout)
            
    return
       

In [89]:
def traitement(liste_tweet:list):
    '''Cette fonction permet de traiter tous les tweets et de les envoyer dans la zone d'atterrissage'''

    for elt in liste_tweet:
        ZoneAtterrissage(elt)
    
    return

In [90]:
traitement(dic)

A présent, tous les tweets sont stockés dans un nouveau fichier json. Ils sont nettoyés et on va pouvoir créer une zone d'entrepot, sous forme de DataFrame. On va y stocker tous les tweets avec seulement les informations dont on à besoin.
On a donc besoin de créer un certain nombre de fonction qui vont nous permettre de récupérer ces informations.

In [2]:
def ListHashtags(tweet):
    ''' Fonction qui renvoie la liste de hashtags d'un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''

    list_hashtags = []

    if "hashtags" in tweet["entities"]:                # on vérifie qu'il y a au moins un hashtag
        for h in range(len(tweet["entities"]["hashtags"])):            # on parcourt la liste des hashtags
            list_hashtags.append(tweet["entities"]["hashtags"][h]["tag"])   

    return list_hashtags

In [3]:
def ListMentionedUsers(tweet):
    ''' Fonction qui renvoie la liste des utilisateurs mentionnés dans un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''

    list_mentioned_users = []

    if "mentions" in tweet["entities"]:                # on vérifie qu'il y a au moins un utilisateur mentionné
        for m in range(len(tweet["entities"]["mentions"])):            # on parcourt la liste des utilisateurs mentionnés
            list_mentioned_users.append(tweet["entities"]["mentions"][m]["username"])

    return list_mentioned_users

In [4]:
def Topic(tweet):
    ''' Fonction qui détermine le topic d'un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''
    pass

In [6]:
def Feeling(tweet):
    ''' Fonction qui détermine le sentiment d'un tweet (positif, négatif ou neutre).
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''
    
    feeling = TextBlob(tweet.get("texte")).sentiment.polarity

    if feeling < 0:
        return "negative"
    elif feeling == 0:
        return "neutral"
    else:
        return "positive"

In [6]:
def ZoneEntrepot(list):
    '''Fonction qui prend en entrée la liste des dictionnaires correspondant
    aux tweets issus du fichier zone_atterrissage et qui créer un DataFrame
    contenant uniquement les informations qui nous intéressent'''

    # Création d'un DataFrame (chaque ligne va correspondre à un tweet)
    zone_entrepot = pd.DataFrame(columns = ['author_id', 'text', 'hashtags', 'mentioned_users', 'topics', 'feelings'])

    for k in range(len(list)-1):        # on travaille sur un tweet à la fois
        # Création d'une liste qui va contenir toutes les informations utiles de ce tweet
        tweet = []
        # Ajout à cette liste des informations récupérées de la zone d'atterissage
        tweet.append(list[k].get("author_id"))              # auteur du tweet    
        tweet.append(list[k].get("text"))                   # texte du tweet
        tweet.append(ListHashtags(list[k]))                 # liste des hashtags
        tweet.append(ListMentionedUsers(list[k]))           # liste des utilisateurs mentionnés
        tweet.append(Topic(list[k]))                        # topic
        tweet.append(Feeling(list[k]))                      # sentiment
        
        # Ajout de la ligne correspondant à ce tweet à notre DataFrame
        zone_entrepot.loc[k+1] = tweet

    return zone_entrepot    

In [1]:
def ListColumn(df, column:str):
    ''' Fonction qui renvoie les éléments d'une colonne d'un DataFrame sous forme d'une liste'''

    return list(df[column])

In [2]:
def NbOccurencesIntoDict(l):
    ''' Fonction qui crée un dictionnaire dans lequel chaque clé
    correspond à un utilisateur, et chaque valeur correspond au nombre
    d'occurrences de cet utilisateur dans la liste'''

    dic = {}

    for elem in l:
        if elem in dic:
            dic[elem] += 1
        else:
            dic[elem] = 1

    return dic

In [3]:
def TopKUsers(df, k):
    ''' Fonction renvoyant les k utilisateurs ayant publié le plus de tweets'''

    l_users = ListColumn(df, "author_id")
    dic_occurences = NbOccurencesIntoDict(l_users)
    #dic_occurences = NbOccurencesIntoDict(ListColumn(df, "author_id"))

    l_sorted = list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))

    return l_sorted[:k]